In [ ]:
#@title  **Community Profiles in the Quranic Conceptual Network**




# --- [الخطوة 0: التثبيت والاستيراد] ---
print("--- [Step 0: Installing Libraries] ---")
!pip uninstall -y community > /dev/null 2>&1
!pip install -qq python-louvain plotly seaborn networkx pandas
!git clone https://github.com/NoorBayan/Amthal.git
%cd Amthal
from utility import *
from IPython.display import display, HTML, clear_output
print("Libraries installed successfully.\n")


# --- [الخطوة 1: الاستيراد] ---
print("--- [Step 1: Importing Libraries] ---")
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import networkx as nx
import community.community_louvain as community_louvain
import seaborn as sns
from itertools import combinations
print("Libraries imported successfully.\n")




# --- [الخطوة 1: تحميل وتجهيز البيانات] ---
print("--- [Step 1: Loading Data] ---")
# ⚠️⚠️⚠️  الرجاء التأكد من أن هذا المسار صحيح في بيئتك
INSTANCES_FILE_PATH = '/content/Amthal/data/processed/instances.csv'
RELATIONS_FILE_PATH = '/content/Amthal/data/processed/relations.csv'
try:
    df = pd.read_csv(INSTANCES_FILE_PATH, sep='\t', encoding='utf-16')
    df['Dominant_Concept']=[Dominant_Concept_dic[i]['ar'] for i in df.Dominant_Concept]
    df['Valence']=[Valence_dic[i]['ar'] for i in df.Valence]
    df['Abstraction_Level']=[Abstraction_Level_dic[i]['ar'] for i in list(df.Abstraction_Level)]

    print(f"Data loaded successfully. Found {len(df)} instances.\n")
except FileNotFoundError:
    print(f"⚠️ ERROR: File not found at '{INSTANCES_FILE_PATH}'. Please check the path and try again.")
    exit()



# ترجمة الأعمدة
valence_map = {'إيجابي': 'Positive', 'سلبي': 'Negative', 'محايد': 'Neutral'}
df['Valence_En'] = df['Valence'].map(valence_map)
abs_level_map = {'محسوس': 'Concrete', 'مجرد': 'Abstract'}
df['Abstraction_Level_En'] = df['Abstraction_Level'].map(abs_level_map)


# --- 2. تعريف المجتمعات بشكل موجه ودمجها مع البيانات ---
print("Defining communities based on thematic analysis and merging with data...")

# هذا هو التقسيم الموضوعي الذي بنيت عليه ورقتك
community_definition = {
    '1. Guidance & Salvation': ['طريق', 'نور', 'مأوى', 'ماء', 'حياة'],
    '2. Power & Judgment': ['قوة', 'خراب', 'خوف', 'نار', 'موت'],
    '3. Cognition & Moral State': ['إدراك', 'ظلام', 'شعور', 'سلوكيات', 'ابتلاء'],
    '4. Social Order & Economy': ['بناء', 'تجارة', 'معاملات', 'خصائص مادية', 'علاقات اجتماعية', 'تطهير وإصلاح']
}

# إنشاء قاموس عكسي لربط كل مفهوم بمجتمعه
concept_to_community_map = {}
for community_name, concepts in community_definition.items():
    for concept in concepts:
        concept_to_community_map[concept] = community_name

# إضافة معلومات المجتمع إلى DataFrame الأصلي
df['Community'] = df['Dominant_Concept'].map(concept_to_community_map)
df.dropna(subset=['Community'], inplace=True) # إزالة أي مثيلات لمفاهيم غير مصنفة


# --- 3. حساب الإحصائيات التجميعية لكل مجتمع ---
print("Aggregating statistics for each predefined community...")
community_profiles = df.groupby('Community').agg(
    Instance_Count=('Instance_ID', 'count'),
    Mean_Dominance_Score=('Dominance_Score', 'mean'),
    Std_Dominance_Score=('Dominance_Score', 'std'),
).reset_index()

valence_counts = pd.crosstab(df['Community'], df['Valence_En'])
valence_profiles = valence_counts.apply(lambda r: f"{r.get('Positive', 0)} / {r.get('Negative', 0)} / {r.get('Neutral', 0)}", axis=1).rename('Valence_Distribution')

abs_level_counts = pd.crosstab(df['Community'], df['Abstraction_Level_En'])
abs_level_profiles = abs_level_counts.apply(lambda r: f"{r.get('Concrete', 0)*100/r.sum():.0f}% / {r.get('Abstract', 0)*100/r.sum():.0f}%", axis=1).rename('Percent_Concrete_Abstract')

community_profiles = community_profiles.merge(valence_profiles, on='Community')
community_profiles = community_profiles.merge(abs_level_profiles, on='Community')


# --- 4. تحديد المفاهيم الأساسية ونوع العلاقة الأكثر شيوعًا ---
print("Identifying core concepts and relation types...")

# الآن، المفاهيم الأساسية هي ببساطة تلك التي عرفناها
community_profiles['Core_Concepts'] = community_profiles['Community'].map(
    lambda name: ', '.join(community_definition.get(name, []))
)

# تحديد نوع العلاقة الأكثر شيوعًا (باستخدام جدول العلاقات الحقيقي)
try:
    relations_df = pd.read_csv(RELATIONS_FILE_PATH, sep='\t', encoding='utf-16')
    df_for_relations = df[['Instance_ID', 'Community']]

    relations_with_comm = relations_df.merge(df_for_relations.rename(columns={'Community': 'Community_1'}),
                                             left_on='Instance_ID_1', right_on='Instance_ID')
    relations_with_comm = relations_with_comm.merge(df_for_relations.rename(columns={'Community': 'Community_2'}),
                                                   left_on='Instance_ID_2', right_on='Instance_ID', suffixes=('_x', ''))

    intra_community_relations = relations_with_comm[relations_with_comm['Community_1'] == relations_with_comm['Community_2']]

    if not intra_community_relations.empty:
        most_common_relation = intra_community_relations.groupby('Community_1')['Relation_Type'].agg(lambda x: x.mode()[0])
        community_profiles = community_profiles.merge(most_common_relation.rename('Most_Common_Relation_Type'),
                                                      left_on='Community', right_index=True, how='left')
        community_profiles['Most_Common_Relation_Type'].fillna('N/A', inplace=True)
    else:
        community_profiles['Most_Common_Relation_Type'] = 'N/A'
except FileNotFoundError:
    print("Warning: Relations file not found. 'Most_Common_Relation_Type' will be N/A.")
    community_profiles['Most_Common_Relation_Type'] = 'N/A'


# --- 5. التنسيق النهائي للجدول ---
print("Formatting the final table...")
community_profiles['Std_Dominance_Score'] = community_profiles['Std_Dominance_Score'].fillna(0)
community_profiles['Mean_Dominance_Score (±SD)'] = community_profiles.apply(
    lambda row: f"{row['Mean_Dominance_Score']:.2f} ± {row['Std_Dominance_Score']:.2f}", axis=1
)
thematic_roles = {
    '1. Guidance & Salvation': 'Positive trajectory toward divine reward',
    '2. Power & Judgment': 'Warning/admonition through divine sovereignty',
    '3. Cognition & Moral State': 'Epistemic and moral battlefield',
    '4. Social Order & Economy': 'Construction of communal and divine-human order'
}
community_profiles['Thematic_Role'] = community_profiles['Community'].map(thematic_roles)

final_columns = [
    'Community', 'Core_Concepts', 'Instance_Count', 'Mean_Dominance_Score (±SD)',
    'Percent_Concrete_Abstract', 'Valence_Distribution', 'Most_Common_Relation_Type',
    'Thematic_Role'
]
final_table = community_profiles[final_columns].sort_values('Community')
final_table.rename(columns={
    'Percent_Concrete_Abstract': '% Concrete / Abstract',
    'Valence_Distribution': 'Valence (Pos/Neg/Neu)',
    'Most_Common_Relation_Type': 'Most Common Relation'
}, inplace=True)


# --- 6. عرض الجدول النهائي ---
print("\n" + "="*120)
print("--- Table Y: Community Profiles in the Quranic Conceptual Network ---")
print("="*120)
print(final_table.to_markdown(index=False))
print("="*120)

# حفظ الداتا فريم كملف CSV
final_table.to_csv("final_table.csv", index=False, encoding="utf-8-sig")

# تنزيل الملف مباشرة على جهازك
from google.colab import files
files.download("final_table.csv")

--- [Step 0: Installing Libraries] ---
Cloning into 'Amthal'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 76 (delta 12), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (76/76), 1.47 MiB | 3.91 MiB/s, done.
Resolving deltas: 100% (12/12), done.
/content/Amthal/Amthal
Libraries installed successfully.

--- [Step 1: Importing Libraries] ---
Libraries imported successfully.

--- [Step 1: Loading Data] ---
Data loaded successfully. Found 4078 instances.

Defining communities based on thematic analysis and merging with data...
Aggregating statistics for each predefined community...
Identifying core concepts and relation types...
Formatting the final table...

--- Table Y: Community Profiles in the Quranic Conceptual Network ---
| Community                  | Core_Concepts                                                    |   Instance_Count | Mean_Dominance_Score (±SD

/tmp/ipython-input-125720017.py:120: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  community_profiles['Most_Common_Relation_Type'].fillna('N/A', inplace=True)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>